In [2]:
import os
import numpy as np
from PIL import Image
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.signal import savgol_filter  # ★ 追加

COLORMAP_NAME = 'jet'          # カラーマップ名
NORM_MODE = 'percentile'       # 'percentile' | 'minmax' | 'fixed'（必要なら分岐を追加）
P_LOW, P_HIGH = 2, 98          # percentile使用時の下限・上限 (%)
RAW_MIN, RAW_MAX = 0, 4095     # 'fixed' 使用時のレンジ（未使用なら無視）

cmap = cm.get_cmap(COLORMAP_NAME)

def hyprawread(name, hor, ver, SpectDim):
    with open(name, 'rb') as f:
        img = np.fromfile(f, np.uint16, -1)
    img = np.reshape(img, (ver, SpectDim, hor))
    img = np.transpose(img, (0, 2, 1))
    return img

# ---- 波長軸ユーティリティ（安全に「最寄りバンド」を引く） ----
def build_wavelengths(start_nm=380, step_nm=5, bands=125):
    return start_nm + step_nm * np.arange(bands, dtype=np.float32)

def nearest_band_index(wavelengths, target_nm: float) -> int:
    return int(np.argmin(np.abs(wavelengths - target_nm)))

# ---- Savitzky–Golayの窓幅を安全に決定（奇数＆バンド数以下） ----
def choose_savgol_params(B: int, base_window=11, polyorder=3):
    # 奇数化
    wl = base_window if base_window % 2 == 1 else base_window + 1
    # バンド数以下に調整（最低7以上を推奨）
    wl = min(wl, B if B % 2 == 1 else B - 1)
    wl = max(7, wl)
    # polyorderは wl-1 より小さく
    po = min(polyorder, wl - 1)
    return wl, po

# ============= メイン処理 =============
for wave in range(400, 1000, 100):  # 例：400, 500, ..., 900 nm を書き出し
    input_directory = '../data/20250905_dataCollection/HSC_data/HSC_data_400nm_1000nm'
    output_directory = f'../data/20250905_dataCollection/HSC_data/processed_images_grad_{wave}nm/'

    os.makedirs(output_directory, exist_ok=True)

    # SIS-Iの場合のパラメータ
    width  = 1200
    height = 1024
    SpectDim = 125

    # 波長配列（例：380, 385, ..., 1000 nm）
    wavelengths = build_wavelengths(start_nm=380, step_nm=5, bands=SpectDim)
    band_idx = nearest_band_index(wavelengths, float(wave))  # ★ 安全に最寄りを取得

    # Savitzky–Golay パラメータ（λ方向に二次微分）
    window_length, polyorder = choose_savgol_params(SpectDim, base_window=11, polyorder=3)  # ★

    for filename in os.listdir(input_directory):
        if not filename.endswith('.nh8'):
            continue

        input_filepath = os.path.join(input_directory, filename)
        img = hyprawread(input_filepath, width, height, SpectDim)  # (H, W, B), uint16想定

        # ---- 必要ならここで反射率化（白基準/ダーク補正）を行う ----
        # 例）img = (img - dark) / (white - dark) ; img = img.clip(0,1)
        # 現状：RAW強度→そのままSGフィルタでも動くが、物理解釈は反射率化が推奨

        # ---- λ方向にSavitzky–Golayで二次微分（各ピクセルのスペクトルに対して）★ ----
        # axis=2 が「波長方向」。float32にしておくと安定
        img_f32 = img.astype(np.float32, copy=False)
        img_diff2 = savgol_filter(
            img_f32, 
            window_length=window_length, 
            polyorder=polyorder, 
            deriv=2, 
            axis=2, 
            mode='interp'  # 端点扱い。必要に応じて'nearest'等
        )

        # ---- 指定波長（最寄りバンド）の「二次微分画像」を取り出す ★ ----
        band = img_diff2[:, :, band_idx]

        # ---- 可視化用の正規化 ----
        if NORM_MODE == 'percentile':
            vmin = np.percentile(band, P_LOW)
            vmax = np.percentile(band, P_HIGH)
            norm = Normalize(vmin=vmin, vmax=vmax, clip=True)
            band_norm = norm(band)
        elif NORM_MODE == 'minmax':
            vmin = np.min(band)
            vmax = np.max(band)
            if vmax > vmin:
                band_norm = (band - vmin) / (vmax - vmin)
            else:
                band_norm = np.zeros_like(band, dtype=np.float32)
        elif NORM_MODE == 'fixed':
            norm = Normalize(vmin=RAW_MIN, vmax=RAW_MAX, clip=True)
            band_norm = norm(band)
        else:
            raise ValueError(f"Unsupported NORM_MODE: {NORM_MODE}")

        rgba = cmap(band_norm)                       # (H, W, 4)
        rgb  = (rgba[..., :3] * 255).astype(np.uint8)
        img_color = Image.fromarray(rgb, mode='RGB')

        save_filename = f"{filename.split('_')[0]}_{filename.split('_')[1]}_d2_{int(wavelengths[band_idx])}nm.png"
        save_filepath = os.path.join(output_directory, save_filename)
        img_color.save(save_filepath)


C:\Users\Atsuya_Watanabe\AppData\Local\Temp\ipykernel_43724\2513592990.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(COLORMAP_NAME)
C:\Users\Atsuya_Watanabe\AppData\Local\Temp\ipykernel_43724\2513592990.py:106: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img_color = Image.fromarray(rgb, mode='RGB')


In [4]:
import os
import numpy as np
from PIL import Image
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.signal import savgol_filter

# ===== 表示設定 =====
COLORMAP_NAME = 'jet'        # 例: 'turbo','viridis','plasma','magma','inferno','cividis','jet'
NORM_MODE = 'percentile'     # 'percentile' | 'minmax' | 'fixed'
P_LOW, P_HIGH = 2, 98        # percentile使用時 (%)
RAW_MIN, RAW_MAX = -0.01, 0.01  # 'fixed' 使用時のレンジ（2次微分は正負が出るので0中心に狭め推奨）

cmap = cm.get_cmap(COLORMAP_NAME)

# ===== ハイパーデータ読み込み =====
def hyprawread(path, hor, ver, SpectDim):
    with open(path, 'rb') as f:
        img = np.fromfile(f, np.uint16, -1)
    img = np.reshape(img, (ver, SpectDim, hor))    # (ver, B, hor)
    img = np.transpose(img, (0, 2, 1))             # -> (H, W, B)
    return img

# ===== 波長ユーティリティ =====
def build_wavelengths(start_nm=900, step_nm=10, bands=81):
    """900, 910, ..., 1700 を想定"""
    return start_nm + step_nm * np.arange(bands, dtype=np.float32)

def nearest_band_index(wavelengths, target_nm: float) -> int:
    return int(np.argmin(np.abs(wavelengths - target_nm)))

def choose_savgol_params(B: int, base_window=11, polyorder=3):
    """窓長は奇数 & バンド数以下、polyorder < window_length"""
    wl = base_window if base_window % 2 == 1 else base_window + 1
    wl = min(wl, B if B % 2 == 1 else B - 1)
    wl = max(7, wl)                      # 最低7推奨
    po = min(polyorder, wl - 1)
    return wl, po

# ===== メイン =====
# 例：900, 1000, 1100, ..., 1700 nm を保存
for wave in range(900, 1701, 100):
    input_directory  = '../data/20250905_dataCollection/HSC_data/HSC_data_1000nm_1700nm'
    output_directory = f'../data/20250905_dataCollection/HSC_data/processed_images_grad2_{wave}nm/'
    os.makedirs(output_directory, exist_ok=True)

    # センサー形状とバンド数（ご環境に合わせて）
    hor, ver = 400, 320
    SpectDim = 81                         # 900–1700 nm (81バンド想定)

    # 波長テーブルと最寄りバンドの確定（安全）
    wavelengths = build_wavelengths(start_nm=900, step_nm=10, bands=SpectDim)
    band_idx = nearest_band_index(wavelengths, float(wave))

    # Savitzky–Golay（λ軸に対して2次微分）
    window_length, polyorder = choose_savgol_params(SpectDim, base_window=11, polyorder=3)

    # フォルダ内の .nir を処理
    for filename in os.listdir(input_directory):
        if not filename.endswith('.nir'):
            continue

        in_path = os.path.join(input_directory, filename)
        cube_u16 = hyprawread(in_path, hor, ver, SpectDim)     # (H, W, B), uint16

        # --- 反射率化が必要な場合はここで (dark/white) を用いて R=(I-dark)/(white-dark) を実施 ---
        cube = cube_u16.astype(np.float32, copy=False)

        # --- λ方向に2次微分（各ピクセルのスペクトルに対して）---
        cube_d2 = savgol_filter(
            cube, window_length=window_length, polyorder=polyorder,
            deriv=2, axis=2, mode='interp'
        )  # (H, W, B), float32

        # --- 指定波長の二次微分画像を抽出 ---
        band_d2 = cube_d2[:, :, band_idx]     # (H, W), float

        # --- 正規化（2次微分は正負あり）---
        if NORM_MODE == 'fixed':
            vmin, vmax = RAW_MIN, RAW_MAX
        elif NORM_MODE == 'minmax':
            vmin, vmax = float(np.min(band_d2)), float(np.max(band_d2))
            if vmax <= vmin:  # 定数画像安全策
                vmin, vmax = -1.0, 1.0
        elif NORM_MODE == 'percentile':
            vmin = float(np.percentile(band_d2, P_LOW))
            vmax = float(np.percentile(band_d2, P_HIGH))
            if vmax <= vmin:
                # 画が潰れるのを避けるため、わずかに広げる
                center = 0.5 * (vmin + vmax)
                vmin, vmax = center - 1e-6, center + 1e-6
        else:
            raise ValueError(f"Unsupported NORM_MODE: {NORM_MODE}")

        norm = Normalize(vmin=vmin, vmax=vmax, clip=True)
        band_norm = norm(band_d2)                 # 0–1

        # --- カラーマップ適用・保存 ---
        rgba = cmap(band_norm)                    # (H, W, 4)
        rgb  = (rgba[..., :3] * 255).astype(np.uint8)
        img_color = Image.fromarray(rgb, mode='RGB')

        # ファイル名：元名の先頭2トークン + d2 + 実バンド波長
        save_filename = f"{filename.split('_')[0]}_{filename.split('_')[1]}_d2_{int(wavelengths[band_idx])}nm.png"
        out_path = os.path.join(output_directory, save_filename)
        img_color.save(out_path)


C:\Users\Atsuya_Watanabe\AppData\Local\Temp\ipykernel_43724\1139867008.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(COLORMAP_NAME)
C:\Users\Atsuya_Watanabe\AppData\Local\Temp\ipykernel_43724\1139867008.py:101: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img_color = Image.fromarray(rgb, mode='RGB')
